# Local uncertainty analysis

In [1]:
from rmgpy.chemkin import *
from rmgpy.tools.canteraModel import *
from rmgpy.quantity import *
from IPython.display import display, Image

In [2]:
# Helper functions for parsing the details out of groups and rate rules from thermo and kinetics



In [3]:
# Load chemkin file
speciesList, reactionList = loadChemkinFile('mech/chem_annotated.inp','mech/species_dictionary.txt')

In [4]:
for species in speciesList:
    print species.thermo.comment
for reaction in reactionList:
    print reaction.kinetics.comment


Thermo library: primaryThermoLibrary


Thermo library: primaryThermoLibrary


Thermo library: primaryThermoLibrary


Thermo library: primaryThermoLibrary


Thermo group additivity estimation: group(Cs-CsHHH) + gauche(Cs(CsRRR)) + other(R) + group(Cs-CsHHH) + gauche(Cs(CsRRR)) + other(R)


Thermo library: primaryThermoLibrary + radical(CH3)


Thermo group additivity estimation: group(Cs-CsHHH) + gauche(Cs(CsRRR)) + other(R) + group(Cs-CsHHH) + gauche(Cs(CsRRR)) + other(R) + radical(CCJ)


Thermo library: primaryThermoLibrary


Thermo library: primaryThermoLibrary


Thermo group additivity estimation: group(Cds-CdsHH) + gauche(CsOsCdSs) + other(R) + group(Cds-CdsHH) + gauche(CsOsCdSs) + other(R)


Thermo library: primaryThermoLibrary


Thermo group additivity estimation: group(Cds-CdsHH) + gauche(CsOsCdSs) + other(R) + group(Cds-CdsHH) + gauche(CsOsCdSs) + other(R) + radical(Cds_P)


Thermo group additivity estimation: group(Cs-CsCsHH) + gauche(Cs(CsCsRR)) + other(R) + group(Cs-CsHHH) +

In [5]:
# Set conditions
ethane = filter(lambda x: x.molecule[0].toSMILES() == 'CC', speciesList)[0]
argon = filter(lambda x: x.molecule[0].toSMILES() == '[Ar]', speciesList)[0]

molFracList=[{ethane: 1}]
Tlist = Quantity([1300,1500,2000],'K')
Plist = Quantity([1],'atm')
reactionTime = Quantity(0.5, 'ms')

In [6]:
# Create cantera object, simulate, and plot
job = Cantera(speciesList=speciesList, reactionList=reactionList)
job.loadChemkinModel('mech/chem_annotated.inp')
job.generateConditions('IdealGasReactor', reactionTime, molFracList, Tlist, Plist)

Converting mech/chem_annotated.inp to /home/connie/Research/Code/rmg_wiki/tools/cantera/chem_annotated.cti...
Wrote CTI mechanism file to '/home/connie/Research/Code/rmg_wiki/tools/cantera/chem_annotated.cti'.
Mechanism contains 19 species and 37 reactions.
Saving into Cantera Model...


In [23]:
canteraReactions = job.model.reactions()
# for reaction in canteraReactions:
# #     print reaction.reactants
# #     print reaction.products
#      print reaction.rate
#      #reaction.rate.temperatureExponent()
from cantera import Arrhenius, NasaPoly2

        
a = Arrhenius(6.18,b=1.0,E=0)
print a
print a.activation_energy
print a.pre_exponential_factor
print a.temperature_exponent


print canteraReactions[1]
print repr(canteraReactions[1])
print canteraReactions[1].rate
canteraReactions[1].rate = a
print canteraReactions[1].rate
    
canteraSpecies = job.model.species()
species = canteraSpecies[0]
print species.thermo
print species.thermo.h(298)
print species.thermo.cp(298)
print species.thermo.s(298)
print type(species.thermo)

# Create a cantera thermo NasaPoly2 object
for species in speciesList:
     #print species.thermo
     print species.thermo.polynomials
     
#nasa = NasaPoly2(300,30000,[])



#canteraSpecies[5].thermo.coeffs  # cannot print coefficients unfortunately

Arrhenius(A=6.18, b=1, E=0)
0.0
6.18
1.0
CH3(2) + ethane(1) <=> C(6) + C2H5(3)
<ElementaryReaction: CH3(2) + ethane(1) <=> C(6) + C2H5(3)>
Arrhenius(A=6.18, b=1, E=0)
Arrhenius(A=6.18, b=1, E=0)
-3117.9232875
20786.15525
154835.26991
<type 'cantera._cantera.NasaPoly2'>
[NASAPolynomial(coeffs=[2.5,0,0,0,0,-745.375,4.37967], Tmin=(200,'K'), Tmax=(1000,'K')), NASAPolynomial(coeffs=[2.5,0,0,0,0,-745.375,4.37967], Tmin=(1000,'K'), Tmax=(6000,'K'))]
[NASAPolynomial(coeffs=[2.5,0,0,0,0,-745.375,0.928724], Tmin=(200,'K'), Tmax=(1000,'K')), NASAPolynomial(coeffs=[2.5,0,0,0,0,-745.375,0.928724], Tmin=(1000,'K'), Tmax=(6000,'K'))]
[NASAPolynomial(coeffs=[2.5,0,0,0,0,-745.375,3.35532], Tmin=(200,'K'), Tmax=(1000,'K')), NASAPolynomial(coeffs=[2.5,0,0,0,0,-745.375,3.35532], Tmin=(1000,'K'), Tmax=(6000,'K'))]
[NASAPolynomial(coeffs=[3.53101,-0.000123661,-5.02999e-07,2.43531e-09,-1.40881e-12,-1046.98,2.96747], Tmin=(200,'K'), Tmax=(1000,'K')), NASAPolynomial(coeffs=[2.95258,0.0013969,-4.92632e-07,7.86